# Textcuboid


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import re
import string
import json
import keras
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

X=X['title']+' '+X['text']

X = X.apply(clean_text)

X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)

X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

to_txt=x_train+x_test

y=y_train+y_test

In [3]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [4]:
#수동으로 cross validation을 하기위한 코드
cnt=5  #cnt가 2일때 두번째 시행
cnt=cnt-2

def exclude_list(input_list, cnt):
    return input_list[:cnt*4000]+input_list[cnt*4000+4000:16000]

if cnt>-1:
    x_train_cnt=exclude_list(x_train,cnt)+x_test
    y_train_cnt=exclude_list(y_train,cnt)+y_test
    x_test_cnt=x_train[cnt*4000:cnt*4000+4000]
    y_test_cnt=y_train[cnt*4000:cnt*4000+4000]

In [5]:
if cnt>-1:
    x_train=x_train_cnt
    y_train=y_train_cnt
    x_test=x_test_cnt
    y_test=y_test_cnt
    to_txt=x_train+x_test
    y=y_train+y_test

## 2) textcuboid 분류

In [ ]:
import random
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from keras.models import load_model

In [6]:
textcuboid=np.load('./1-Channel textcuboid_WELFake(bert).npy')

In [7]:
textcuboid_test=np.load('./1-Channel textcuboid_test_WELFake(bert).npy')

In [8]:
tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [11]:
input_layer = Input(shape=(53, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)
pooling = Dropout(0.5)(pooling)
x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 53, 768)]         0         
                                                                 
 conv1d (Conv1D)             (None, 53, 1024)          787456    
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                             

In [14]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

In [15]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
59/59 [==============================] - 2s 17ms/step - loss: 1.9613 - accuracy: 0.5366 - val_loss: 0.6853 - val_accuracy: 0.6710 - lr: 0.0010
Epoch 2/20
59/59 [==============================] - 1s 13ms/step - loss: 0.6192 - accuracy: 0.6539 - val_loss: 0.4526 - val_accuracy: 0.8180 - lr: 0.0010
Epoch 3/20
59/59 [==============================] - 1s 14ms/step - loss: 0.4295 - accuracy: 0.8055 - val_loss: 0.3063 - val_accuracy: 0.8770 - lr: 0.0010
Epoch 4/20
59/59 [==============================] - 1s 14ms/step - loss: 0.3238 - accuracy: 0.8655 - val_loss: 0.2945 - val_accuracy: 0.8820 - lr: 0.0010
Epoch 5/20
59/59 [==============================] - 1s 14ms/step - loss: 0.2712 - accuracy: 0.8894 - val_loss: 0.2229 - val_accuracy: 0.9090 - lr: 0.0010
Epoch 6/20
59/59 [==============================] - 1s 14ms/step - loss: 0.2252 - accuracy: 0.9107 - val_loss: 0.2039 - val_accuracy: 0.9210 - lr: 0.0010
Epoch 7/20
59/59 [==============================] - 1s 13ms/step - loss: 0.1

In [16]:
y_test=np.array(y_test)

In [17]:
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 2ms/step - loss: 0.2194 - accuracy: 0.9377


[0.21942150592803955, 0.937749981880188]

In [20]:
model = load_model('best_model_10.h5')
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 2ms/step - loss: 0.1684 - accuracy: 0.9330


[0.1684194952249527, 0.9330000281333923]